In [16]:
from __future__ import unicode_literals, print_function, division
import unicodedata
import numpy as np
import pandas as pd
from io import open
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import random
import wandb
import re

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
wandb.login()

True

## DATA PROCESSING

In [18]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.chr2index = {}
        self.chr2count = {}
        self.index2chr = {0: "$", 1: "#"}
        self.n_chrs = 2 

    def addword(self, word):
        for ch in word:
            self.addchr(ch)

    def addchr(self, ch):
        if ch not in self.chr2index:
            self.chr2index[ch] = self.n_chrs
            self.chr2count[ch] = 1
            self.index2chr[self.n_chrs] = ch
            self.n_chrs += 1
        else:
            self.chr2count[ch] += 1

In [19]:
MAX_LENGTH = 25
# def unicodeToAscii(s):
#     return ''.join(
#         c for c in unicodedata.normalize('NFD', s)
#         if unicodedata.category(c) != 'Mn'
#     )


# def normalizeString(s):
#     s = unicodeToAscii(s.strip())
#     s = re.sub(r"([.!?])", r" \1", s)
#     s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
#     return s

In [20]:

def readLangs(lang1,lang2):
    pairs = ()
    for i in ['train','valid','test']:
        fd = open('aksharantar_sampled/'+lang2+'/'+lang2+'_'+i+'.csv')
        lines = fd.read().strip().split('\n')
        pairs += ([[s for s in l.split(',')] for l in lines],)
    return pairs
    

In [21]:
input_lang = Lang("eng")
output_lang = Lang("mni")
train_pairs,valid_pairs,test_pairs = readLangs("eng","mni")

for pair in train_pairs:
    input_lang.addword(pair[0])
    output_lang.addword(pair[1])
    

In [22]:
def indexesFromWord(lang, word):
    return [lang.chr2index[ch] for ch in word]


def tensorFromWord(lang,word):
    indexes = indexesFromWord(lang, word)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromWord(input_lang, pair[0])
    target_tensor = tensorFromWord(output_lang, pair[1])
    return (input_tensor, target_tensor)

## ENCODER

In [23]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_layer_size, num_encoder_layers, cell_type, dropout_prob, bidirectional):
      super(EncoderRNN, self).__init__()
      self.input_size = input_size
      self.hidden_layer_size = hidden_layer_size
      self.num_encoder_layers = num_encoder_layers
      self.cell_type = cell_type

      self.embedding = nn.Embedding(self.input_size, embedding_size)

      if cell_type == 'RNN':
        self.rnn = nn.RNN(input_size = embedding_size, hidden_size = hidden_layer_size, num_layers = num_encoder_layers, dropout = dropout_prob, bidirectional = bidirectional)
      elif cell_type == 'LSTM':
        self.rnn = nn.LSTM(input_size = embedding_size, hidden_size = hidden_layer_size, num_layers = num_encoder_layers, dropout = dropout_prob, bidirectional = bidirectional)
      elif cell_type == 'GRU':
        self.rnn = nn.GRU(input_size = embedding_size, hidden_size = hidden_layer_size, num_layers = num_encoder_layers, dropout = dropout_prob, bidirectional = bidirectional)
      self.dropout = nn.Dropout(dropout_prob)
      self.D = 1
      if bidirectional == True :
        self.D = 2

    def forward(self, input_tensor, prev_hidden, prev_cell = None):
      embedded = self.embedding(input_tensor).view(1,1,-1)
      embedded = self.dropout(embedded)
      if self.cell_type == 'RNN':
        output, hidden = self.rnn(embedded,prev_hidden)
      elif self.cell_type == 'LSTM':
        output, (hidden,cell) = self.rnn(embedded,(prev_hidden,prev_cell))
        return output, (hidden,cell)
      elif self.cell_type == 'GRU':
        output, hidden = self.rnn(embedded,prev_hidden)
        
      return output,hidden

    def initHidden(self):
      if self.cell_type == 'LSTM':
        hidden = torch.zeros(self.D*self.num_encoder_layers,1,self.hidden_layer_size,device = device)
        cell = torch.zeros(self.D*self.num_encoder_layers,1,self.hidden_layer_size,device = device)
        return hidden,cell
      else :
        hidden = torch.zeros(self.D*self.num_encoder_layers,1,self.hidden_layer_size,device = device)
      return hidden


## DECODER

In [24]:
# class Decoder

class DecoderRNN(nn.Module):
  def __init__(self, output_size, embedding_size, hidden_layer_size, num_decoder_layers, cell_type, dropout_prob, bidirectional):
    super(DecoderRNN, self).__init__()
    self.output_size = output_size
    self.hidden_layer_size = hidden_layer_size
    self.num_decoder_layers = num_decoder_layers
    self.cell_type = cell_type
    self.embedding_size = embedding_size
    self.embedding = nn.Embedding(output_size, embedding_size)
   
    if cell_type == 'RNN':
      self.rnn = nn.RNN(input_size = embedding_size, hidden_size = hidden_layer_size, num_layers = num_decoder_layers, dropout = dropout_prob, bidirectional = bidirectional)
    elif cell_type == 'LSTM':
      self.rnn = nn.LSTM(input_size = embedding_size, hidden_size = hidden_layer_size, num_layers = num_decoder_layers, dropout = dropout_prob, bidirectional = bidirectional)
    elif cell_type == 'GRU':
      self.rnn = nn.GRU(input_size = embedding_size, hidden_size = hidden_layer_size, num_layers = num_decoder_layers, dropout = dropout_prob, bidirectional = bidirectional)
      
    self.D = 1
    if bidirectional == True :
      self.D = 2
      
    self.dropout = nn.Dropout(dropout_prob)
    self.out = nn.Linear(self.D*hidden_layer_size, output_size)
    self.softmax = nn.LogSoftmax(dim = 1)
      
  def forward(self, input_tensor, prev_hidden, prev_cell = None):
    embedded = self.embedding(input_tensor).view(1,1,-1)
    embedded = F.relu(embedded)
    embedded = self.dropout(embedded)
    
    if self.cell_type == 'RNN':
      output, hidden = self.rnn(embedded,prev_hidden)
      
    elif self.cell_type == 'LSTM':
      output,(hidden,cell) = self.rnn(embedded,(prev_hidden,prev_cell))
    
    elif self.cell_type == 'GRU':
      output, hidden = self.rnn(embedded,prev_hidden)

    output = self.softmax(self.out(output[0]))

    if self.cell_type == 'LSTM':
      return output,(hidden,cell)
    
    return output, hidden

In [25]:
def train(input_tensor,target_tensor,encoder,decoder,encoder_optimizer,decoder_optimizer,criterion,max_length = MAX_LENGTH,teacher_forcing_ratio = 0.5):
    
    if encoder.cell_type == 'LSTM':
      encoder_hidden,encoder_cell = encoder.initHidden()
    else :
      encoder_hidden = encoder.initHidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    encoder_outputs = torch.zeros(max_length, encoder.D*encoder.hidden_layer_size, device=device)
    
    loss = 0
    
    for ei in range(input_length):
      if encoder.cell_type == 'LSTM':
        encoder_output,(encoder_hidden,encoder_cell) = encoder(input_tensor = input_tensor[ei],prev_hidden = encoder_hidden,prev_cell = encoder_cell)
      else :
        encoder_output, encoder_hidden = encoder(input_tensor = input_tensor[ei], prev_hidden = encoder_hidden)
      # print(encoder_output.size())
      encoder_outputs[ei] = encoder_output[0][0]
      
    decoder_input = torch.tensor([[SOS_token]],device=device)
    # decoder_outputs = torch.zeros(target_length)
    decoder_hidden = encoder_hidden
    if decoder.cell_type == 'LSTM':
      decoder_cell = encoder_cell
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing :
      for di in range(target_length):
        if decoder.cell_type == 'LSTM':
          decoder_output,(decoder_hidden,decoder_cell) = decoder(input_tensor = decoder_input,prev_hidden = decoder_hidden,prev_cell = decoder_cell)
        else:
          decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden)
        loss += criterion(decoder_output, target_tensor[di])
        decoder_input = target_tensor[di]

    else :
      for di in range(target_length):
        if decoder.cell_type == 'LSTM':
          decoder_output,(decoder_hidden,decoder_cell) = decoder(input_tensor = decoder_input,prev_hidden = decoder_hidden,prev_cell = decoder_cell)
        else:
          decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()
        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == EOS_token:
          break
    
    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length
        
      

In [26]:
def trainIters(encoder,decoder,encoder_optimizer,decoder_optimizer,criterion):
    
    training_pairs = [tensorsFromPair(pair) for pair in train_pairs]
    training_loss = 0
    it = 0
    for pair in training_pairs:
        input_tensor = pair[0]
        target_tensor = pair[1]
        loss = train(input_tensor,target_tensor,encoder,decoder,encoder_optimizer,decoder_optimizer,criterion)
        training_loss += loss
        if it%200==0:
            print(it,"done")
        it += 1
    return training_loss/len(training_pairs)

In [27]:
def evaluate(encoder,decoder,word,target = None,criterion = None,max_length = MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromWord(input_lang,word)
        input_length = input_tensor.size()[0]
        target_length = 0
        if target != None:
            target_tensor = tensorFromWord(output_lang,target)
            target_length = target_tensor.size(0)
        
        loss = 0
        
        if encoder.cell_type == 'LSTM':
            encoder_hidden,encoder_cell = encoder.initHidden()
        else :
            encoder_hidden = encoder.initHidden()
            
        encoder_outputs = torch.zeros(max_length,encoder.D*encoder.hidden_layer_size,device=device)
        
        for ei in range(input_length):
            if encoder.cell_type == 'LSTM':
                encoder_output,(encoder_hidden,encoder_cell) = encoder(input_tensor = input_tensor[ei],prev_hidden = encoder_hidden,prev_cell = encoder_cell)
            else :
                encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]
        
        decoder_input = torch.tensor([[SOS_token]],device=device)
        decoder_hidden = encoder_hidden
        if decoder.cell_type == 'LSTM':
            decoder_cell = encoder_cell
            
        decoded_word = ''
        

        for di in range(max_length):
            if decoder.cell_type == 'LSTM':
                decoder_output,(decoder_hidden,decoder_cell) = decoder(input_tensor = decoder_input,prev_hidden = decoder_hidden,prev_cell = decoder_cell)
            else:
                decoder_output,decoder_hidden = decoder(decoder_input,decoder_hidden)
            topv, topi = decoder_output.topk(1)
            
            if di<target_length:
                loss += criterion(decoder_output,target_tensor[di])
                
            if topi.item() == EOS_token:
                decoded_word += '#'
                break
            else :
                decoded_word += output_lang.index2chr[topi.item()]
                
            decoder_input = topi.squeeze().detach()
        
        return decoded_word,loss
            
def evaluateRandomly(encoder,decoder,n=10):
    for i in range(n):
        pair = random.choice(train_pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_word = evaluate(encoder, decoder, pair[0])
        print('<', output_word)
        print('')

def eval_acc(encoder,decoder,pairs,criterion = None):
    count = 0
    tot_loss = 0
    for pair in pairs:
        pred_word,loss = evaluate(encoder,decoder,pair[0],target=pair[1],criterion=criterion)
        tot_loss += loss
        if pred_word[:-1] == pair[1] : count += 1
    return float(count/len(pairs)),tot_loss

def run(encoder,decoder,encoder_optimizer,decoder_optimizer,criterion,epochs):
    for epoch in range(epochs): 
        train_loss = trainIters(encoder=encoder,decoder=decoder,encoder_optimizer=encoder_optimizer,decoder_optimizer=decoder_optimizer,criterion = criterion)
        train_acc,_ = eval_acc(encoder,decoder,random.sample(train_pairs,1000),criterion)
        valid_acc,valid_loss = eval_acc(encoder,decoder,valid_pairs,criterion)
        print("Epoch : %d, Training Loss : %f, Training Accuracy : %f, Validation Loss = %f, Validation Accuracy : %f" % (epoch+1,train_loss,train_acc,valid_loss,valid_acc)) 
        wandb.log({ "training_accuracy" : train_acc,
                    "validation_accuracy" : valid_acc,
                    "training_loss" : train_loss,
                    "validation_loss" : valid_loss,
                    "epoch" : epoch+1})
        

## Training 

In [28]:
# input_size = input_lang.n_chrs
# output_size = output_lang.n_chrs
# embedding_size = 64
# hidden_layer_size = 64
# num_layers = 2
# cell_type = 'LSTM'
# dropout_prob = 0.05
# learning_rate = 0.01
# bidirectional = False
# epochs = 15

# encoder = EncoderRNN(input_size=input_size,embedding_size=embedding_size,hidden_layer_size=hidden_layer_size,num_encoder_layers=num_layers,cell_type=cell_type,dropout_prob=dropout_prob,bidirectional=bidirectional).to(device)
# decoder = DecoderRNN(output_size=output_size,embedding_size=embedding_size,hidden_layer_size=hidden_layer_size,num_decoder_layers=num_layers,cell_type=cell_type,dropout_prob=dropout_prob,bidirectional=bidirectional).to(device)
# encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
# decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
# criterion = nn.NLLLoss()
# for epoch in range(epochs): 
#     train_loss = trainIters(encoder=encoder,decoder=decoder,encoder_optimizer=encoder_optimizer,decoder_optimizer=decoder_optimizer,criterion = criterion)
#     train_acc,_ = eval_acc(encoder,decoder,train_pairs[0:10],criterion)
#     valid_acc,valid_loss = eval_acc(encoder,decoder,valid_pairs[0:10],criterion)
#     print("Epoch : %d, Training Loss : %f, Trainging Accuracy : %f, Validation Loss = %f, Validation Accuracy : %f" % (epoch+1,train_loss,train_acc,valid_loss,valid_acc)) 

In [29]:
def train_model():
    config_defaults = {
        "embedding_size" : 256,
        "hidden_layer_size" : 256,
        "num_layers" : 2,
        "cell_type" : 'GRU',
        "dropout_prob" : 0.1,
        "learning_rate" : 0.001,
        "bidirectional" : False,
        "epochs" : 10,
    }
    
    wandb.init(config=config_defaults,dir='/home/arunesh/.local/lib/python3.8/site-packages')
    config = wandb.config
    input_size = input_lang.n_chrs
    output_size = output_lang.n_chrs
    embedding_size = config.embedding_size
    hidden_layer_size = config.hidden_layer_size
    num_layers = config.num_layers
    cell_type = config.cell_type
    dropout_prob = config.dropout_prob
    learning_rate = config.learning_rate
    bidirectional = config.bidirectional
    epochs = config.epochs
    encoder = EncoderRNN(input_size=input_size,embedding_size=embedding_size,hidden_layer_size=hidden_layer_size,num_encoder_layers=num_layers,cell_type=cell_type,dropout_prob=dropout_prob,bidirectional=bidirectional).to(device)
    decoder = DecoderRNN(output_size=output_size,embedding_size=embedding_size,hidden_layer_size=hidden_layer_size,num_decoder_layers=num_layers,cell_type=cell_type,dropout_prob=dropout_prob,bidirectional=bidirectional).to(device)
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()
    print("started")
    run(encoder,decoder,encoder_optimizer,decoder_optimizer,criterion,epochs)
               
    run_name = "es_{}_hl_{}_nl_{}_ct_{}_dp_{}_lr_{}_bi_{}_ep_{}".format(embedding_size,hidden_layer_size,num_layers,cell_type,dropout_prob,learning_rate,bidirectional,epochs)
    wandb.run.name = run_name
    wandb.run.save()
    

In [30]:
sweep_config = {
    # "name" : "assignment_sweeps",
    "method" : "bayes",
    "metric" :{
        "name" : "validation_accuracy",
        "goal" : "maximize"
    },
    "parameters" : {
        "embedding_size" : {
            "values" : [128,256]
        },
        "num_layers" : {
            "values" : [2,3,4]
        },
        "hidden_layer_size" : {
            "values" : [128,256]
        },
        "learning_rate" : {
            "values" : [5e-3,1e-3]
        },
        "cell_type" : {
           "values" : ['LSTM', 'GRU'] 
        },
        "dropout" : {
            "values" : [0.1,0.2]
        },
        "bidirectional": {
            "values" : [True,False]
        },
        "epochs": {
            "values" : [10]
        }
    }
}


In [ ]:
sweep_id = wandb.sweep(sweep_config,project="Assignment_3")
wandb.agent(sweep_id = sweep_id,function = train_model,count = 25)

Create sweep with ID: 1nprjcog
Sweep URL: https://wandb.ai/cs20b009/Assignment_3/sweeps/1nprjcog


wandb: Agent Starting Run: nc50zsd4 with config:
wandb: 	bidirectional: False
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.2
wandb: 	embedding_size: 128
wandb: 	epochs: 10
wandb: 	hidden_layer_size: 128
wandb: 	learning_rate: 0.005
wandb: 	num_layers: 2
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
2023-05-17 18:48:54.249631: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 18:48:54.534558: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-17 18:48:54.53

started
0 done
200 done
400 done
600 done
800 done
1000 done
1200 done
1400 done
1600 done
1800 done
2000 done
2200 done
2400 done
2600 done
2800 done
3000 done
3200 done
3400 done
3600 done
3800 done
4000 done
4200 done
4400 done
4600 done
4800 done
5000 done
5200 done
5400 done
5600 done
5800 done
6000 done
6200 done
6400 done
6600 done
6800 done
7000 done
7200 done
7400 done
7600 done
7800 done
8000 done
8200 done
8400 done
8600 done
8800 done
9000 done
9200 done
9400 done
9600 done
9800 done
10000 done
Epoch : 1, Training Loss : 1.916255, Training Accuracy : 0.045000, Validation Loss = 40871.492188, Validation Accuracy : 0.072086
0 done
200 done
400 done
600 done
800 done
1000 done
1200 done
1400 done
1600 done
1800 done
2000 done
2200 done
2400 done
2600 done
2800 done
3000 done
3200 done
3400 done
3600 done
3800 done
4000 done
4200 done
4400 done
4600 done
4800 done
5000 done
5200 done
5400 done
5600 done
5800 done
6000 done
6200 done
6400 done
6600 done
6800 done
7000 done
7200 

wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Epoch : 10, Training Loss : 0.882003, Training Accuracy : 0.309000, Validation Loss = 28585.304688, Validation Accuracy : 0.282209


...

epoch,▁▂▃▃▄▅▆▆▇█
training_accuracy,▄▃▂▃▁▃▃▄▃█
training_loss,█▄▃▃▃▂▂▂▁▁
validation_accuracy,▆▅▇▆▁▃▇█▄█
validation_loss,▃▅▃▅█▅▂▂▄▁
epoch,10
training_accuracy,0.017
training_loss,1.73939
validation_accuracy,0.01687
validation_loss,40344.07422


wandb: Agent Starting Run: jqb9ru1n with config:
wandb: 	bidirectional: True
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.1
wandb: 	embedding_size: 128
wandb: 	epochs: 10
wandb: 	hidden_layer_size: 256
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 4
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
2023-05-18 02:44:05.165438: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-18 02:44:05.308060: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-18 02:44:05.308

started
0 done
200 done
400 done
600 done
800 done
1000 done
1200 done
1400 done
1600 done
1800 done
2000 done
2200 done
2400 done
2600 done
2800 done
3000 done
3200 done
3400 done
3600 done
3800 done
4000 done
4200 done
4400 done
4600 done
4800 done
5000 done
5200 done
5400 done
5600 done
5800 done
6000 done
6200 done
6400 done
6600 done
6800 done
7000 done
7200 done
7400 done
7600 done
7800 done
8000 done
8200 done
8400 done
8600 done
8800 done
9000 done
9200 done
9400 done
9600 done
9800 done
10000 done
Epoch : 1, Training Loss : 2.115265, Training Accuracy : 0.015000, Validation Loss = 43504.816406, Validation Accuracy : 0.022393
0 done
200 done
400 done
600 done
800 done
1000 done
1200 done
1400 done
1600 done
1800 done
2000 done
2200 done
2400 done
2600 done
2800 done
3000 done
3200 done
3400 done
3600 done
3800 done
4000 done
4200 done
4400 done
4600 done
4800 done
5000 done
5200 done
5400 done
5600 done
5800 done
6000 done
6200 done
6400 done
6600 done
6800 done
7000 done
7200 

wandb: Ctrl + C detected. Stopping sweep.


epoch,▁▂▃▅▆▇█
training_accuracy,▁▄▅▆███
training_loss,█▅▃▂▁▁▁
validation_accuracy,▁▄▆▇███
validation_loss,█▄▂▂▁▁▁
epoch,7
training_accuracy,0.723
training_loss,0.32137
validation_accuracy,0.59571
validation_loss,15688.34961


In [32]:
wandb.finish()